<a href="https://colab.research.google.com/github/PomesHouse/SkillTreePython-DataAnalysis/blob/main/04.%ED%83%90%EC%83%89%EC%A0%81%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D/%EC%99%80%EC%9D%B8%ED%92%88%EC%A7%88%EB%B6%84%EB%A5%98EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 와인 품질 분류 EDA
---

## 라이브러리 및 데이터 로드

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-muted')

In [ ]:
train = pd.read_csv('/content/train.csv')
train.shape

In [ ]:
train.head()

## 데이터 기본 확인

#### 결측치 확인

In [ ]:
print(train.isnull().sum()) # 결측치 없음

#### 기술통계

In [ ]:
del train['index'] # 'Index' column은 따로 필요하지 않기에 제거 
train.info() # type을 제외한 모든 변수가 수치형 데이터

In [ ]:
train.describe()

In [ ]:
train.describe(include = 'object')

In [ ]:
train['type'].value_counts()

#### 변수 요약 

In [ ]:
fig, axes = plt.subplots(3, 5, figsize = (18, 10))
axes = axes.flatten()

for col, ax in zip(train.columns, axes):
    sns.histplot(data = train, x = col, ax = ax)
plt.show()

In [ ]:
nums = ['fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol'] # 연속형
noms = ['type'] # 이산형 
y = 'quality' # 예측값

* 데이터 정리
    * 데이터 소개: 와인 성분 데이터
    * 자료형 소개
     * 범주 자료형
        * type: 와인 종류
     * 이산 자료형 
        * quality: 품질 (예측해야할 값)
     * 연속 자료형 
        * fixed acidity: 산도
        * volatile acidity: 휘발성산
        * citric acid: 시트르산
        * residual sugar: 잔당(발효 후 와인 속에 남아있는 당)
        * chlorides: 염화물
        * free sulfur dioxide: 독립 이산화황
        * total sulfur dioxide: 총 이산화황
        * density: 밀도
        * pH: 수소이온농도
        * sulphates: 황산염
        * alcohol: 도수



## 데이터 분포

#### 데이터 분포 시각화

In [ ]:
# 연속 자료형 분포 

fig, axes = plt.subplots(3, 4, figsize = (18, 10))
axes = axes.flatten()

for col, ax in zip(nums, axes):
    sns.histplot(data = train, x = col, ax = ax)
plt.show()

In [ ]:
# 이산 자료형 분포

sns.histplot(data = train, x = 'type')

#### 기술 통계량 시각화

In [ ]:
# 기술 통계량 확인
train.describe()

In [ ]:
# boxplot으로 연속 자료형 기술 통계 시각화  

fig, axes = plt.subplots(3, 4, figsize = (18, 15))
axes = axes.flatten()

for col, ax in zip(nums, axes):
    sns.boxplot(data = train, x = 'quality', y = col, ax = ax)
    ax.set_title(f'skew = {np.round(train[col].skew(),2)}   kurtosis = {np.round(train[col].kurt(),2)}')
plt.show()

#### 예측값에 대한 추가 시각화

In [ ]:
sns.countplot(data = train, x = 'quality')
plt.show()

In [ ]:
sns.countplot(data = train, x = 'quality', hue = 'type')
plt.show()

## 속성간 관계 파악 

#### 상관분석

In [ ]:
plt.figure(figsize = (10, 5))
sns.heatmap(data = train.corr(), annot = True, vmax = 1, vmin = -1, cmap = 'coolwarm')
plt.xticks(rotation = 45)
plt.show()

quality와 상관관계가 abs(0.1) 이상인 속성
* volatile acidity (음의 상관관계)
* chlorides (음의 상관관계)
* density (음의 상관관계)
* alcohol (양의 상관관계)
 * 이 중에서 alcohol과의 상관관계가 가장 크다.

quality와는 별개로 상관관계가 abs(0.5) 이상인 속성
* total sulfur dioxide vs. free sulfur dioxide (양의 상관관계)
* density vs. residual sugar (양의 상관관계)
* alcohol vs. density (음의 상관관계)


In [ ]:
# 비교적 quality와의 상관관계가 높았던 속성
hi = ['volatile acidity','chlorides', 'density', 'alcohol']

#### 속성간 관계 시각화 

In [ ]:
sns.pairplot(data = train[['quality'] + hi + ['type']].sample(200), hue = 'quality')
plt.show()

In [ ]:
for col in hi:
    sns.displot(data = train, x = col, kind = 'kde', hue = 'quality', col = 'type')

* 육안으로 그래프를 보았을 때, 그나마 상관관계가 높았던 속성으로도 quality 구분은 힘들어보인다. 
* 레드와인과 화이트와인으로 구분해 각 속성의 분포를 보아도 크게 눈에 띄는 패턴을 보이지 않는다. 
* 와인의 품질은 한 속성의 극단성에 의해서 결정되는 것이 아니라, 여러 속성들의 전체적인 밸런스에 의해 좌우되어서 일 수 있다. 

In [ ]:
plt.subplot(131)
sns.scatterplot(data = train, x = 'alcohol', y = 'volatile acidity', hue = 'quality')
plt.subplot(132)
sns.scatterplot(data = train, x = 'alcohol', y = 'chlorides', hue = 'quality')
plt.subplot(133)
sns.scatterplot(data = train, x = 'alcohol', y = 'density', hue = 'quality')
plt.show()

In [ ]:
sns.lmplot(data = train, x = 'alcohol', y = 'volatile acidity', hue = 'quality', col ='quality')
sns.lmplot(data = train, x = 'alcohol', y = 'chlorides', hue = 'quality', col = 'quality')
sns.lmplot(data = train, x = 'alcohol', y = 'density', hue = 'quality', col = 'quality')
plt.show()

* 위 그래프에서도 여전히 품질별로 뚜렷한 구분을 하기가 어렵다.

In [ ]:
_ = sns.histplot(data = train, x = 'quality', y = 'type')

* 품질이 6이었을 경우 화이트와인일 확률이 높다. 

## 결론


*  Data visualization으로 quality와 특정 속성의 관계를 파악하는데는 한계가 있다. 
 * 연속 자료형 속성의 경우 quality와 alcohol의 상관관계가 가장 높았으나, 그마저도 quality의 구분을 하기에 가시적인 정보를 주지 않았다. 
 * 범주 자료형 속성인 type의 경우 quality == 6인 경우 화이트와인인 경우가 많았으나, quality가 높아지거나 낮아질 수록 type에 상관 없는 모습을 보인다.  
* 이는 위에서 언급했듯이, 와인의 품질이라는 것이 한 화학적 속성의 극단성으로만 결정되지 않기 때문일 것이라 추측된다. 
* 후에 진행될 모델링과 훈련의 역할이 클 것이므로, 머신러닝의 용이함을 위해 데이터 전처리 과정에서 모든 수치 자료형에게 min-max scaling 또는 standard(z-score) scaling을 해준다.  
* 와인 등급의 경우 0,1,2... 의 숫자로 스케일링 해준다. 
* 와인 type의 경우 0,1로 바꾸어준다. 